In [8]:
# work in progres (neveiks)
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, classification_report
from itertools import permutations,combinations,product
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier

In [9]:
max_depth = [1,2,3]
max_width = [5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

combinations = []

for depth in max_depth:
    pro = [list(p) for p in product(max_width, repeat=depth)]
    combinations.append(list(pro))


In [10]:
first = True
for comb in combinations:
    for c in comb:
        # ann carcass
        ann = tf.keras.models.Sequential()
        for nodes in c:
            ann.add(tf.keras.layers.Dense(units=nodes, activation='relu'))
        ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
        ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
        # name of ann
        name = str(c)
        print(name)
        # checkpoints
        checkpoint_path ="ANN_files/" + name + "/cp.ckpt"
        checkpoint_dir = os.path.dirname(checkpoint_path)
        cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)
        # training
        times = 0
        
        for chunk in pd.read_csv(r'Data\train_data.csv', chunksize=500000, low_memory = False):
            if times > 3:
                break
            X = chunk.iloc[:, 0:-1].values
            y = chunk.iloc[:, -1].values
            ann.partial_fit(X, y, batch_size = 32, epochs = 100,callbacks=[cp_callback])

            times_test = 0
            true_cm = [[0,0],[0,0]]
            for chunk in pd.read_csv(r'Data\test_data.csv', chunksize=50000, low_memory = False):

                X_test = chunk.iloc[:, 0:-1].values
                y_test = chunk.iloc[:, -1].values

                if times_test > 3:
                    break

                y_pred = ann.predict(X_test)
                y_pred = (y_pred > 0.5)

                cm = confusion_matrix(y_test, y_pred)

                true_cm[0] = cm[0] + true_cm[0]
                true_cm[1] = cm[1] + true_cm[1]
                true_cm[2] = cm[2] + true_cm[2]
                true_cm[3] = cm[3] + true_cm[3]
                
                times_test = times_test + 1

            
            print(true_cm)

            times = times + 1 


[5]


AttributeError: 'Sequential' object has no attribute 'partial_fit'

In [ ]:

Ch_size = 700000



for chunk in pd.read_csv(r'Data\test_data.csv', chunksize=Ch_size):
    X_train = chunk.iloc[:, 0:-1].values
    y_train = chunk.iloc[:, -1].values

    suby = []
    for i in range(len(y_train)):
        if y_train[i] == 'Benign':
            suby.append(0)
        else:
            suby.append(1)
    y = suby.copy()
    del suby
    
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)

    print(chunk)
    break
# X = dataset.iloc[:, 3:-1].values
# y = dataset.iloc[:, -1].values
# le = LabelEncoder()
# X[:, 2] = le.fit_transform(X[:, 2])

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)